<a href="https://colab.research.google.com/github/jimin0/Graduation_Project/blob/main/ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Huelse/SEAL-Python.git
%cd SEAL-Python

# Install dependencies
!pip3 install numpy pybind11

# Init the SEAL and pybind11
!git submodule update --init --recursive
# Get the newest repositories (dev only)
# git submodule update --remote

# Build the SEAL lib
%cd SEAL
!cmake -S . -B build -DSEAL_USE_MSGSL=OFF -DSEAL_USE_ZLIB=OFF
!cmake --build build
%cd ..

# Run the setup.py
!python3 setup.py build_ext -i


Cloning into 'SEAL-Python'...
remote: Enumerating objects: 1628, done.
remote: Counting objects: 100% (252/252), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 1628 (delta 175), reused 191 (delta 162), pack-reused 1376
Receiving objects: 100% (1628/1628), 8.66 MiB | 21.26 MiB/s, done.
Resolving deltas: 100% (879/879), done.
/content/SEAL-Python/SEAL-Python
Submodule 'SEAL' (https://github.com/microsoft/SEAL.git) registered for path 'SEAL'
Submodule 'pybind11' (https://github.com/pybind/pybind11.git) registered for path 'pybind11'
Cloning into '/content/SEAL-Python/SEAL-Python/SEAL'...
Cloning into '/content/SEAL-Python/SEAL-Python/pybind11'...
Submodule path 'SEAL': checked out '206648d0e4634e5c61dcf9370676630268290b59'
Submodule path 'pybind11': checked out 'be97c5a98b4b252c524566f508b5c79410d118c6'
/content/SEAL-Python/SEAL-Python/SEAL
-- Build type (CMAKE_BUILD_TYPE): Release
-- The CXX compiler identification is GNU 11.4.0
-- The C compiler identification is G

In [ ]:
# SEAL 라이브러리의 모든 클래스와 함수를 임포트합니다.
from seal import *
import numpy as np

# BGV 암호화 스킴을 사용할 암호화 매개변수 객체를 생성합니다.
parms = EncryptionParameters(scheme_type.bgv)

# 다항식 모듈러스의 차수를 설정합니다.
# 이는 암호화된 다항식의 최대 차수를 정의하고,
# 암호화의 보안 수준과 성능에 영향을 미칩니다.
"""
poly_modulus_degree가 클수록 암호화된 텍스트의 크기가 커지고 모든 연산이 느려지지만,
더 복잡한 암호화된 계산을 가능하게 함. 권장되는 값들은 1024, 2048, 4096, 8192, 16384, 32768이지만,
이 범위를 넘어가는 것도 가능
"""
poly_modulus_degree = 8192
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))
parms.set_plain_modulus(PlainModulus.Batching(poly_modulus_degree, 20))


# 설정된 암호화 매개변수를 기반으로 SEAL 컨텍스트를 생성합니다.
# 이 컨텍스트는 암호화, 복호화, 키 생성 등에 사용됩니다.
context = SEALContext(parms)


# 계수 모듈러스를 설정합니다. 이는 암호화 시 사용되는 모듈러스 값으로,
# BFVDefault는 보안 수준에 맞는 기본 모듈러스 값을 선택합니다.
parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))

# 평문 모듈러스를 설정합니다.
# 이 모듈러스는 암호화되지 않은 데이터가 취할 수 있는 값의 범위를 정의합니다.
# 이 경우, 값의 범위는 0에서 255입니다.
parms.set_plain_modulus(256)



# """
# plaintext modulus는 양의 정수여야 합니다.
# plaintext modulu는 plaintext 데이터 타입의 크기를 결정하며, 곱셈에서 노이즈 예산의 소비를 결정합.
# 따라서, 최상의 성능을 위해 가능한 한 plaintext 데이터 타입을 작게 유지하는 것이 중요함.

# (!) 각각의 암호화된 값은  noise budget 을 가지고 있음. 암호문에 대한 연산을 할 때마다  noise budget이 소모됨.
# 한 암호문의 노이즈 예산이 0에 도달하면 그 암호문은 너무 손상되어 복호화할 수 없게 된다함.

# """


In [ ]:
# 키 생성

keygen = KeyGenerator(context)

secret_key = keygen.secret_key()
public_key = keygen.create_public_key()
relin_keys = keygen.create_relin_keys()
galois_keys =  keygen.create_galois_keys()

encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)


batch_encoder = BatchEncoder(context)
slot_count = batch_encoder.slot_count()
row_size = slot_count / 2
print(f'Plaintext matrix row size: {row_size}')

Plaintext matrix row size: 4096.0



TODO
*   곱셈 결과 범위가 256이하인 연산 테스트해보기
*   연속 계산할 경우 correctness 확인
*   2*3*4*2*1*... 이렇게 곱셈을 연달아서 여러번 하는 경우 --> bootstrapping 필요할 수 있음





In [ ]:
# x = 2 암호화

x="2"
x_plain = Plaintext(x)

# plain text로 변경
print("intput : x")
print(f'x_plain: {x_plain}')
print("Express x = " + x + " as a plaintext polynomial " +
      x_plain.to_string() + ".")
print(f'x_plain type: {type(x_plain)}')

print()

# 암호화
x_encrypted1 = Ciphertext()
print("암호화")
x_encrypted1 = encryptor.encrypt(x_plain)
print(f'x_encrypted: {x_encrypted1}')
print(f'x_encrypted to string: {x_encrypted1.to_string()}')


# 암호화 된거 정보 출력
print("    + type of encrypted1: ", type(x_encrypted1))
print("    + size of freshly encrypted1 x: " + str(x_encrypted1.size()))
print("    + Coefficient Modulus Size:",str(x_encrypted1.coeff_modulus_size()))
print("    + poly_modulus_degree Size:",str(x_encrypted1.poly_modulus_degree()))
# 노이즈 예산 출력
print("    + noise budget in freshly encrypted x: " +
      str(decryptor.invariant_noise_budget(x_encrypted1)) + " bits")

print()

# 복호화
print("복호화")
x_decrypted = Plaintext()
decryptor.decrypt(x_encrypted1,x_decrypted)
print(f'x_decrypted: {x_decrypted}')
print(f'x_decrypted to string: {x_decrypted.to_string()}')
print()



# 원복 비교
print("비교")
if x == x_decrypted.to_string():
    print(f'Correct!:  [x:{x}] == [x_degrypted: {x_decrypted.to_string()}]')
else:
    print(f'Incorrect!:  [x:{x}] != [x_degrypted: {x_decrypted.to_string()}]')

# --------------------------------------------------------------
print("-"*50)
"""
2 x 3

암호문 * 암호문을 해보겠다. + 재선형화

"""
expected_result = str(2*3)

print("intput : 3")
print("식 : 2 x 3")

print("\n암호화")
x_encrypted2 = encryptor.encrypt(Plaintext("3"))
print(f'x_encrypted: {x_encrypted2}')
#print(f'x_encrypted to string: {x_encrypted.to_string()}')

# 암호화 된거 정보 출력
print("    + size of freshly encrypted x: " + str(x_encrypted2.size()))
print("    + Coefficient Modulus Size:",str(x_encrypted2.coeff_modulus_size()))
print("    + poly_modulus_degree Size:",str(x_encrypted2.poly_modulus_degree()))
# 노이즈 예산 출력
print("    + noise budget in freshly encrypted x: " +
      str(decryptor.invariant_noise_budget(x_encrypted2)) + " bits")
print()

# 연산
print("연산")

# 먼저 결과를 저장할 Ciphertext 객체를 생성
twoPthree = Ciphertext()

# 암호문끼리 곱셈을 수행하고, 결과를 'destination'에 저장
twoPthree = evaluator.multiply(x_encrypted1, x_encrypted2)
evaluator.relinearize_inplace(twoPthree, relin_keys)
print("2 x 3" + " as a ciphertext " + str(twoPthree) + ".") #destination
print("    + After addition, size of twoMthree: " + str(twoPthree.size()))
print("    + noise budget after addition: " + str(decryptor.invariant_noise_budget(twoPthree)) + " bits")
print("    + type of twoMthree: ", type(twoPthree))

# 복호화
print("\n복호화")
x_decrypted = Plaintext()
decryptor.decrypt(twoPthree,x_decrypted)
print(f'x_decrypted: {x_decrypted}')
print(f'x_decrypted to string: {x_decrypted.to_string()}')
print()

# 원복 비교
print("비교")
if expected_result == x_decrypted.to_string():
    print(f'Correct!:  [x:{expected_result}] == [x_degrypted: {x_decrypted.to_string()}]')
else:
    print(f'Incorrect!:  [x:{expected_result}] != [x_degrypted: {x_decrypted.to_string()}]')

# --------------------------------------------------------------



print("-"*50)
"""
2 x 3 x 4

암호문 * plain(4)

"""

print("intput : 4")
print("식 : 2 x 3 x 4")

number_to_multiply = 4
list_to_encode = [number_to_multiply]
batch_encoder = BatchEncoder(context)

plain_four = Plaintext()
plain_four = batch_encoder.encode(list_to_encode)


print("    + size of freshly x_encrypted4 x: " + str(twoPthree.size()))
print("    + noise budget in freshly encrypted x: " +
      str(decryptor.invariant_noise_budget(twoPthree)) + " bits")

# 암호화된 데이터 연산 (2 x 3 x 4)
sixPfour = Ciphertext()
sixPfour = evaluator.multiply_plain(twoPthree, plain_four)
print("    + noise budget in freshly tewnty4Pfive: " +
      str(decryptor.invariant_noise_budget(sixPfour)) + " bits")


decrypted_result = Plaintext()
decryptor.decrypt(sixPfour, decrypted_result)
decoded_result = batch_encoder.decode(decrypted_result)

# 결과 확인
expected_result = 2 * 3 * 4  # 24
if decoded_result[0] == expected_result:
    print(f'Correct!: [Expected: {expected_result}] == [Decrypted: {decoded_result[0]}]')
else:
    print(f'Incorrect!: [Expected: {expected_result}] != [Decrypted: {decoded_result[0]}]')



# # --------------------------------------------------------------
print("-"*50)
"""
2 x 3 x 4 x 2

암호문 * 암호문

"""

print("intput : 2")
print("식 : 2 x 3 x 4 x 2")

number_to_multiply = 2
list_to_encode = [number_to_multiply]
batch_encoder = BatchEncoder(context)

plain_two = Plaintext()
plain_two = batch_encoder.encode(list_to_encode)

x_encrypted4 = Ciphertext()
x_encrypted4 = encryptor.encrypt(plain_two)

print("    + size of freshly x_encrypted4 x: " + str(x_encrypted4.size()))
print("    + Coefficient Modulus Size:",str(x_encrypted4.coeff_modulus_size()))
print("    + poly_modulus_degree Size:",str(x_encrypted4.poly_modulus_degree()))
print("    + noise budget in freshly encrypted x: " +
      str(decryptor.invariant_noise_budget(x_encrypted4)) + " bits")

# 암호화된 데이터 연산 (2 x 3 x 4 x 2)
tewnty4Ptwo = Ciphertext()
tewnty4Ptwo = evaluator.multiply(sixPfour, x_encrypted4)
evaluator.relinearize_inplace(tewnty4Ptwo, relin_keys)
print("    + noise budget in freshly tewnty4Pfive: " +
      str(decryptor.invariant_noise_budget(tewnty4Ptwo)) + " bits")

# 복호화 및 결과 확인
decrypted_result = Plaintext()
decryptor.decrypt(tewnty4Ptwo, decrypted_result)
decoded_result = batch_encoder.decode(decrypted_result)



# 결과 확인
expected_result = 2 * 3 * 4 * 2 # 48
if decoded_result[0] == expected_result:
    print(f'Correct!: [Expected: {expected_result}] == [Decrypted: {decoded_result[0]}]')
else:
    print(f'Incorrect!: [Expected: {expected_result}] != [Decrypted: {decoded_result[0]}]')


# # --------------------------------------------------------------
print("-"*50)
"""
2 x 3 x 4 x 2 * 3 """



print("intput : 3")
print("식 : 2 x 3 x 4 x 2 x 3")

number_to_multiply = 3
list_to_encode = [number_to_multiply]
batch_encoder = BatchEncoder(context)

plain_three = Plaintext()
plain_three = batch_encoder.encode(list_to_encode)

x_encrypted5 = Ciphertext()
x_encrypted5 = encryptor.encrypt(plain_three)


print("    + noise budget in freshly encrypted x: " +
      str(decryptor.invariant_noise_budget(x_encrypted5)) + " bits")

# 암호화된 데이터 연산 (2 x 3 x 4 x 5)
tewnty4Pthee = Ciphertext()
tewnty4Pthee = evaluator.multiply(tewnty4Ptwo, x_encrypted5)
evaluator.relinearize_inplace(tewnty4Pthee, relin_keys)
print("    + noise budget in freshly tewnty4Pfive: " +
      str(decryptor.invariant_noise_budget(tewnty4Pthee)) + " bits")

# 복호화 및 결과 확인
decrypted_result = Plaintext()
decryptor.decrypt(tewnty4Pthee, decrypted_result)
decoded_result = batch_encoder.decode(decrypted_result)



# 결과 확인
expected_result = 2 * 3 * 4 * 2 * 3
if decoded_result[0] == expected_result:
    print(f'Correct!: [Expected: {expected_result}] == [Decrypted: {decoded_result[0]}]')
else:
    print(f'Incorrect!: [Expected: {expected_result}] != [Decrypted: {decoded_result[0]}]')



# # --------------------------------------------------------------
print("-"*50)
"""
2 x 3 x 4 x 2 * 3  * 1"""



print("intput : 1")
print("식 : 2 x 3 x 4 x 2 x 3 * 1")

number_to_multiply = 1
list_to_encode = [number_to_multiply]
batch_encoder = BatchEncoder(context)

plain_one= Plaintext()
plain_one = batch_encoder.encode(list_to_encode)

x_encrypted6 = Ciphertext()
x_encrypted6 = encryptor.encrypt(plain_one)


print("    + noise budget in freshly encrypted x: " +
      str(decryptor.invariant_noise_budget(x_encrypted6)) + " bits")

# 암호화된 데이터 연산 (2 x 3 x 4 x 5)
t144 = Ciphertext()
t144 = evaluator.multiply(tewnty4Pthee, x_encrypted6)
evaluator.relinearize_inplace(t144, relin_keys)
print("    + noise budget in freshly tewnty4Pfive: " +
      str(decryptor.invariant_noise_budget(t144)) + " bits")

# 복호화 및 결과 확인
decrypted_result = Plaintext()
decryptor.decrypt(t144, decrypted_result)
decoded_result = batch_encoder.decode(decrypted_result)



# 결과 확인
expected_result = 2 * 3 * 4 * 2 * 3 * 1
if decoded_result[0] == expected_result:
    print(f'Correct!: [Expected: {expected_result}] == [Decrypted: {decoded_result[0]}]')
else:
    print(f'Incorrect!: [Expected: {expected_result}] != [Decrypted: {decoded_result[0]}]')

intput : x
x_plain: <seal.Plaintext object at 0x7f32112d0670>
Express x = 2 as a plaintext polynomial 2.
x_plain type: <class 'seal.Plaintext'>

암호화
x_encrypted: <seal.Ciphertext object at 0x7f32112d0230>
x_encrypted to string: b'^\xa1\x10\x04\x01\x02\x00\x00\xbb\x99\x06\x00\x00\x00\x00\x00(\xb5/\xfd\xa0a\x00\x08\x00\x9c\x15\rn\xfd\xdf\xa7h*\x10\xa8\x1d\xe60\xa6J,\xd6\xa6\x07\x02\x03\xc1L}\xb3\x8f\xb8\xbeP2\xe0<R4\x02\x8e[\x1fo\xdej\xd9\xff\x81RJ)\xa5L\x01\x96h\x83h\xa8h\xab\x95\x90!v(x\x96\xf2\xac\x94\x95+\xbf\x8a\xb9\xe8\x0bC\xe7\xdfH%,"3\xbf^mZ\xf4\xe8\xc9\xb7\x9eR:\xcc\x11\xbeK)\xe9\x82\xda\x93\xff\x85\x8f\n_9>}\x93D\x13\xd5\xa8O=Y\x1fi$\x9f\xa5b\x01\xd9\xadx\xf6N\x8f\xa1\xb4F\x7f\x8b\x1a\x9f\x8a_\xbf\x86\x92*E\x85\xd0\xb3d\xd0\xe6\xb04\xf8w\xf4\x8c\x07\xdf\xe3\xbf!\xe6^\xbf\xadwex\x01C\xc4\xebW\xcb;\x17|\xca|*i\x80\xa1\x83<\xdf\xfay$G\xa3\xc1\xaf\x9c\x18Hh\xde\xf9V\xd4\x1dB\t\xc7\xb7w\xc6@*\x00\xe4\xdf\xe3\x81S\xe0\xe2\xa7\xed)GA\n_MS\xa6e\x06\xea\xd3\xb1I\xc0=J~M\xf2\xc0\xc8\x8b\

In [ ]:
# --------------------------------------------------------------
print("-"*50)
"""
2 x 3 x 4 x 5 x 1

암호문 * 암호문

"""


print("intput : 1")
print("식 : 2 x 3 x 4 x 2 x 1")

number_to_multiply = 1
list_to_encode = [number_to_multiply]
batch_encoder = BatchEncoder(context)

plain_one = Plaintext()
plain_one = batch_encoder.encode(list_to_encode)

x_encrypted5 = Ciphertext()
x_encrypted5 = encryptor.encrypt(plain_five)

print("    + size of freshly x_encrypted3 x: " + str(x_encrypted5.size()))
print("    + Coefficient Modulus Size:",str(x_encrypted5.coeff_modulus_size()))
print("    + poly_modulus_degree Size:",str(x_encrypted5.poly_modulus_degree()))
print("    + noise budget in freshly encrypted x: " +
      str(decryptor.invariant_noise_budget(x_encrypted5)) + " bits")

# 암호화된 데이터 연산 (2 x 3 x 4 x 2 * 1)
onehundPone = Ciphertext()
onehundPone = evaluator.multiply(tewnty4Pfive, x_encrypted5)
print("    + noise budget in freshly onehundPone: " +
      str(decryptor.invariant_noise_budget(onehundPone)) + " bits")

# 복호화 및 결과 확인
decrypted_result = Plaintext()
decryptor.decrypt(onehundPone, decrypted_result)
decoded_result = batch_encoder.decode(decrypted_result)



# 결과 확인
expected_result = 2 * 3 * 4 * 5 * 1  # 120
if decoded_result[0] == expected_result:
    print(f'Correct!: [Expected: {expected_result}] == [Decrypted: {decoded_result[0]}]')
else:
    print(f'Incorrect!: [Expected: {expected_result}] != [Decrypted: {decoded_result[0]}]')


print("-"*50)
print("intput : 1")
print("식 : 2 x 3 x 4 x 5 x 1")

# 올바른 암호화 객체 사용
plain_one = Plaintext()
plain_one = batch_encoder.encode([1])
x_encrypted6 = Ciphertext()
x_encrypted6 = encryptor.encrypt(plain_one)

# 암호화된 데이터 연산 (2 x 3 x 4 x 5 x 1)
onehundPone = Ciphertext()
onehundPone = evaluator.multiply(SixPfour, x_encrypted6) # where SixPfour is the result of 2 x 3 x 4 x 5
evaluator.relinearize_inplace(onehundPone, relin_keys)

# 복호화 및 결과 확인
decrypted_result = Plaintext()
decryptor.decrypt(onehundPone, decrypted_result)
decoded_result = batch_encoder.decode(decrypted_result)

# 결과 확인
expected_result = 2 * 3 * 4 * 5 * 1  # 120
if decoded_result[0] == expected_result:
    print(f'Correct!: [Expected: {expected_result}] == [Decrypted: {decoded_result[0]}]')
else:
    print(f'Incorrect!: [Expected: {expected_result}] != [Decrypted: {decoded_result[0]}]')


--------------------------------------------------
intput : 1
식 : 2 x 3 x 4 x 5 x 1
    + size of freshly x_encrypted3 x: 2
    + Coefficient Modulus Size: 4
    + poly_modulus_degree Size: 8192
    + noise budget in freshly encrypted x: 145 bits
    + noise budget in freshly onehundPone: 0 bits
Incorrect!: [Expected: 120] != [Decrypted: 62015]
--------------------------------------------------
intput : 1
식 : 2 x 3 x 4 x 5 x 1
Incorrect!: [Expected: 120] != [Decrypted: 24]


## Plaintext 인코딩 실험 테스트

In [ ]:
from seal import *
# Plaintext 객체 생성
plain_four_str = Plaintext("4")
print("Plaintext content:", plain_four_str.to_string())

#BatchEncoder 생성
batch_encoder = BatchEncoder(context)

# 인코딩할 데이터
values_to_encode = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # 예시 데이터
plain = Plaintext()
plain_test = batch_encoder.encode(values_to_encode)

# 암호화
encrypted = Ciphertext()
encrypted_test = encryptor.encrypt(plain_test)

# 복호화 및 디코딩
decrypted = Plaintext()
decryptor.decrypt(encrypted_test, decrypted)
decoded_values = []
decoded_values = batch_encoder.decode(decrypted)

# 결과 출력
print("Original values:", values_to_encode)
print("Decoded values:", decoded_values)



# 생성된 Plaintext 객체의 내용 출력 : 문자열 4
print("Plaintext content using '4':", plain_four_str, type(plain_four_str))

# BatchEncoder를 사용한 결과 출력 : int 4
print("BatchEncoder encoded content:", values_to_encode[3], type(decoded_values[3]))


# 다항식 내부 출력
print("Plaintext content using '4':", plain_four_str.to_string())
print("BatchEncoder encoded content:", plain_test.to_string())

Plaintext content: 4
Original values: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Decoded values: [1 2 3 ... 0 0 0]
Plaintext content using '4': <seal.Plaintext object at 0x7d7bfd96f0f0> <class 'seal.Plaintext'>
BatchEncoder encoded content: 4 <class 'numpy.int64'>
Plaintext content using '4': 4
BatchEncoder encoded content: 3824x^8191 + 44A76x^8190 + A0017x^8189 + 8DE98x^8188 + 6CA9Ex^8187 + 2F23x^8186 + 3BD84x^8185 + CDC00x^8184 + EECF8x^8183 + 97DFCx^8182 + 72535x^8181 + CA076x^8180 + 730CAx^8179 + E609Dx^8178 + B0B6x^8177 + 51E5x^8176 + D413Bx^8175 + 2A5BEx^8174 + 1BBAx^8173 + 45CABx^8172 + 8A7Ex^8171 + 323CDx^8170 + AE820x^8169 + ED277x^8168 + 703C1x^8167 + 38A18x^8166 + 1960Bx^8165 + F07BAx^8164 + 8F12Bx^8163 + 2C1FBx^8162 + 9EB6Ex^8161 + 3ACBDx^8160 + C32CBx^8159 + 99E66x^8158 + 51178x^8157 + F050Bx^8156 + C8C03x^8155 + 9BD5Cx^8154 + B6FBAx^8153 + C922x^8152 + 806DFx^8151 + AD78Dx^8150 + 8D241x^8149 + 4A2C8x^8148 + 255F8x^8147 + 25B3Fx^8146 + 1A7FCx^8145 + D9E63x^8144 + E4C8Ax^8143 + D3154x

In [ ]:
def print_vector(vector):
    print('[ ', end='')
    for i in range(0, 8):
        print(vector[i], end=', ')
    print('... ]')


인수로 안하고 리스트에 저장해서 암호화해보기

In [ ]:
# 길이가 slot_count 길이인 리스트 생성하고 0으로 초기화
pod_matrix = [0] * slot_count

# 값 할당
pod_matrix[0] = 2
pod_matrix[1] = 3
pod_matrix[2] = 4
pod_matrix[3] = 2
pod_matrix[4] = 1
pod_matrix[5] = 3
pod_matrix[6] = 1.5
pod_matrix[7] = 3



print(f'pod matrix: {pod_matrix}')
print(f'len pod matrix : {len(pod_matrix)}')  #8192
print()

#plain text로 인코딩
x_plain = batch_encoder.encode(pod_matrix)

# x_plain의 내용을 문자열로 변환하여 출력
print(f'x_plain: {x_plain}')
print(f'x_plain to string: {x_plain.to_string()}')
print()

#암호화
x_encrypted = encryptor.encrypt(x_plain)
print(f'x_encryted : {x_encrypted}')
print("Encoded encrypt text:", x_encrypted.to_string())
print(f'noise budget in freshly encrypted x: {decryptor.invariant_noise_budget(x_encrypted)}') # 146
print('-'*50)



# for i in range(1, len(encrypted_elements)):
#   temp_result = Ciphertext()
#   temp_result = evaluator.multiply(result_multi, encrypted_elements[i])
#   evaluator.relinearize_inplace(temp_result, relin_keys)
#   result_multi = temp_result

# # Decrypt and decode the result
# decrypted_result = decryptor.decrypt(result_multi)
# decoded_result = batch_encoder.decode(decrypted_result)

# # Print the final result
# print(f'Decoded result: {decoded_result}')

# x_multiply = evaluator.square(x_encrypted)

# print(f'size of x_squared: {x_squared.size()}')

# print()
# evaluator.relinearize_inplace(x_squared, relin_keys)
# print(f'size of x_squared (after relinearization): {x_squared.size()}')
# print(f'noise budget in x_squared: {decryptor.invariant_noise_budget(x_squared)} bits')
# decrypted_result = decryptor.decrypt(x_squared)
# pod_result = batch_encoder.decode(decrypted_result)
# print_vector(pod_result)
# print('-'*50)

pod matrix: [2, 3, 4, 2, 1, 3, 1.5, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
# x_encrypted의 제곱과 유사하지만, 서로 다른 두 개의 객체를 곱함
x_multiplied = evaluator.multiply(x_encrypted, )

# 재선형화 수행
evaluator.relinearize_inplace(x_multiplied, relin_keys)

# 결과 크기 및 잡음 예산 출력
print(f'size of x_multiplied: {x_multiplied.size()}')
print(f'noise budget in x_multiplied: {decryptor.invariant_noise_budget(x_multiplied)} bits')

# 복호화 및 결과 출력
decrypted_result_multiplied = decryptor.decrypt(x_multiplied)
pod_result_multiplied = batch_encoder.decode(decrypted_result_multiplied)
print_vector(pod_result_multiplied)


size of x_multiplied: 2
noise budget in x_multiplied: 110 bits
[ 4, 9, 16, 4, 1, 0, 0, 0, ... ]


In [ ]:
encrypted_values = [encryptor.encrypt(batch_encoder.encode([2])),
                    encryptor.encrypt(batch_encoder.encode([3])),
                    encryptor.encrypt(batch_encoder.encode([4])),
                    encryptor.encrypt(batch_encoder.encode([2]))]
result_encrypted = evaluator.multiply_many(encrypted_values, relin_keys)


In [ ]:
result_decrypted = decryptor.decrypt(result_encrypted)
result_decoded = batch_encoder.decode(result_decrypted)
print(result_decoded)


[48  0  0 ...  0  0  0]


## 리스트 가능한지 테스트

In [ ]:
# 리스트 데이터
data_list = [1, 2, 3, 4]

# 인코딩 및 암호화
plain = batch_encoder.encode(data_list)
encrypted = encryptor.encrypt(plain)

# 동형 연산 수행 (예: 제곱)
squared = evaluator.square(encrypted)
evaluator.relinearize_inplace(squared, relin_keys)

# 복호화 및 디코딩
decrypted = decryptor.decrypt(squared)
decoded_result = batch_encoder.decode(decrypted)

print(decoded_result)


[1 4 9 ... 0 0 0]


## tuple 가능한지 테스트

In [ ]:
data_tuple = (1, 2, 3, 4)

# Encode and encrypt
plain = batch_encoder.encode(data_tuple)
encrypted = encryptor.encrypt(plain)

# Perform operations (e.g., square)
squared = evaluator.square(encrypted)
evaluator.relinearize_inplace(squared, relin_keys)

# Decrypt and decode
decrypted = decryptor.decrypt(squared)
decoded_result = batch_encoder.decode(decrypted)

# Convert list back to tuple if needed
result_tuple = tuple(decoded_result)

print(result_tuple)

(1, 4, 9, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

##  Numpy 사용 가능한지 test
-> 1차원만 가능

In [ ]:
# NumPy 배열 생성
data_array = np.array([1, 2, 3, 4])
array_2x2 = np.array([[1, 2], [3, 4]])

# NumPy 배열을 리스트로 변환
flattened_list = array_2x2.flatten().tolist()

# 인코딩 및 암호화
plain = batch_encoder.encode(flattened_list)
encrypted = encryptor.encrypt(plain)

# 동형 연산 수행 (예: 제곱)
squared = evaluator.square(encrypted)
evaluator.relinearize_inplace(squared, relin_keys)

# 복호화 및 디코딩
decrypted = decryptor.decrypt(squared)
decoded_result = batch_encoder.decode(decrypted)

# 결과를 NumPy 배열로 변환
result_array = np.array(decoded_result)
print(result_array)


[1 4 9 ... 0 0 0]
